# PublicDatasets

## 1. Introduction

### 1.1 Info

**Current Pipeline:**

* First pick targets
* Find appropriate input
* Run Downloader
* Read PDFs
* Query for Relations
* Save Results

#### 1.1.1 Target Venues

* MIDL 2021
  * Proceedings to use: https://proceedings.mlr.press/v143/

#### 1.1.2 Target Datasets

* [Data Science Bowl 2017](https://www.kaggle.com/c/data-science-bowl-2017)
  * See here for literature: https://scholar.google.com/scholar?hl=en&as_sdt=0%2C5&q=%22data+science+bowl%22+%2B+lung&btnG=

### 1.2 Imports

In [2]:
import os,sys
import scrapy

## 2. Input

### 2.1 PDF Sources

In [4]:
# These are configured under `ArticleScraper/ArticleScraper/spiders`

### 2.2 Explicit Mentions 

In [5]:
mentions = [
    "2017 Data Science Bowl",
    "Kaggle Data Science Bowl 2017",
    "Data Science Bowl 2017",
    "KDSB17",
    "DSB",
]

### 2.3 Related Keywords

In [6]:
keywords = [
    "lung cancer",
    "nodule",
    "competition",
    "kaggle dataset",
    "deep learning"
]

## 3. Downloader

In [ ]:
# scrapy code goes here
# dois
# pdf-files

## 4. PDF-Reader

## 5. Relation Querier

## 6. Saving Results